In [2]:
import numpy as np
import pandas as pd
from decimal import Decimal, getcontext
import time
import copy

# Part 0 -- Loading/Transforming our Data

In [9]:
players_df = pd.read_csv("DKSalaries_11_16_21.csv")
players_df.head()

,Position,Name + ID,Name,ID,Roster Position,Salary,Game Info,TeamAbbrev,AvgPointsPerGame
0,LW,Alex Ovechkin (20206545),Alex Ovechkin,20206545,W/UTIL,9100,WAS@LA 11/17/2021 10:30PM ET,WAS,20.73
1,C,Nathan MacKinnon (20206385),Nathan MacKinnon,20206385,C/UTIL,8800,COL@VAN 11/17/2021 09:00PM ET,COL,14.45
2,C,Mikko Rantanen (20206387),Mikko Rantanen,20206387,C/UTIL,8300,COL@VAN 11/17/2021 09:00PM ET,COL,13.80
3,G,Darcy Kuemper (20207005),Darcy Kuemper,20207005,G,8100,COL@VAN 11/17/2021 09:00PM ET,COL,14.05
4,G,Jonas Johansson (20207006),Jonas Johansson,20207006,G,8100,COL@VAN 11/17/2021 09:00PM ET,COL,10.10


In [10]:
## We just used the first letter for the position
players_df["Position"] = players_df["Roster Position"].str[0]

## Remove rows we don't use
players_df = players_df.drop(columns=["Name + ID","ID","Game Info","TeamAbbrev", "Roster Position"])

## Convert our data from a dataframe to a list of lists
data_for_algo = players_df[["Name", "Salary", "AvgPointsPerGame", "Position"]].values.tolist()
data_for_algo[0]


['Alex Ovechkin', 9100, 20.73, 'W']

In [11]:
def printTable(table):
    '''A simple way to display matrices in a more readable format'''
    for i in table:
        print(i)

# Part 1: Building a dynamic programming algorithm that gets an optimal roster of palyers

In [12]:
def get_optimal_set(players_list, budget, inc, limits):
    '''Uses a Dynamic Programming Algorithm to get the maximum number of points scored on a fantasy hockey team,
    constrained by salary cap (budget) and limits on positions (only 3 centers, 2 wings, etc.)'''
    
    ## inputs
    # players_list--a list of lists: player name, salary, expected points, position--ex:['Nathan MacKinnon', 9000, 14.94, 'C']
    # budget -- a total budget/salary cap--ex:50000
    # inc -- an increment that you go up/down by when you use the algorithm--ex:100 
    # limits -- a dictionary of constraints for each position--ex:{"C": 2,"W": 3 ,"D" : 2,"G": 1}
    
    ## first we divide all working money sums by the increment
    ## This allows us to control the granularity of the algorith
    players = [[player[0], int(int(player[1])/inc), player[2], player[3]] for player in players_list]
    budget = int(budget / inc)


    ## we get our number of players and create dictionaries for our combo table and our combo traceback table
    nPlayers = len(players) 
    combos = {}
    combos_traceback = {}
    
    ## A cassic napsack problem is only measuring 2 variables (weight and value), making it a nested list solution
    ## since this has a limitation of how many of each positions you can have, we are making a nest list solution
    ## for every possible combination of position limits
    
    ## We iterate through every possible combination of centers, wings, defenders, and goalies
    for centers in range(limits["C"] + 1):
        for wings in range(limits["W"] + 1):
            for defenders in range(limits["D"] + 1):
                for goalies in range(limits["G"] + 1):
                    
                    ## For each combination, we initialize a list of lists of placeholder values                 
                    combos[(centers, wings, defenders, goalies)] = [[-1 for i in range(budget + 1)] for i in range(nPlayers + 1)]
                    combos_traceback[(centers, wings, defenders, goalies)] = [[None for i in range(budget + 1)] for i in range(nPlayers + 1)]

                    
    ## Now that we have created players/budget tables for each combination of positions,
    ## we iterate through through each combination to start calculating our best draft option    
    for centers in range(limits["C"] + 1):
        for wings in range(limits["W"] + 1):
            for defenders in range(limits["D"] + 1):
                for goalies in range(limits["G"] + 1):
                
                
                    roster_limits = {"C":centers,"W": wings ,"D":defenders ,"G": goalies}
                    table = combos[(centers, wings, defenders, goalies)]
                    traceback = combos_traceback[(centers, wings, defenders, goalies)]
                
                    ## Filling in the base case for 0 players to choose from
                    for i in range(budget + 1):
                        table[0][i] = 0
                    
                    ## Now we incrememnt through each player
                    for player_row in range(1, nPlayers + 1):
                        
                        player_cost = players[player_row - 1][1]
                        player_points = players[player_row - 1][2]
                        player_position = players[player_row -1][3]
                        
                        
                        ## For each player, we increment through our budget
                        for budget_interval in range(budget + 1):
                        
                            ##For each acceptable player, we see if we can afford the player and if we have roster space
                            if roster_limits[player_position] > 0 and budget_interval >= player_cost:
                                
                                
                                ## To determine if we want to select a player or not, we find the higher value between:
                                ## The ROI of the player + the optimized ROI of our remaining budget and available positions
                                ## The ROI of not selecting the player
                                
                                ## Since these are tables of previously filled in dictionaries,
                                ## We just have to go to the right player limit combination, player row, and budget interval
                                ## to get the already-calculated optimized ROI
                                if player_position == "C":
                                    restOfROI = combos[(centers - 1, wings, defenders, goalies)][player_row - 1][budget_interval - player_cost]
                                elif player_position == "W":
                                    restOfROI = combos[(centers, wings -1 , defenders, goalies)][player_row - 1][budget_interval - player_cost]
                                elif player_position == "D":
                                    restOfROI = combos[(centers, wings, defenders - 1, goalies)][player_row - 1][budget_interval - player_cost]
                                elif player_position == "G":
                                    restOfROI = combos[(centers, wings, defenders, goalies - 1)][player_row - 1][budget_interval - player_cost]
                                    

                                ## We compare it for the optimal ROI if we dont sign the player
                                noSign = table[player_row - 1][budget_interval]
                
                                ## if signing the player generates more profit than not signing the player, then we add him to the table
                                if player_points + restOfROI > noSign:
                                    table[player_row][budget_interval] = player_points + restOfROI
                                    traceback[player_row][budget_interval] = True
                                
                                else:
                                    table[player_row][budget_interval] = noSign
                                    traceback[player_row][budget_interval] = False
                                    
                            ## if we cant afford the player or don't have roster room, we treat it as a noSign
                            else:
                                table[player_row][budget_interval] = table[player_row - 1][budget_interval]
                                traceback[player_row][budget_interval] = False
                                
                                
                                
    
    ## To return the optimized investment portfolio, we create an empty return list
    ## and initialize our item marker value, and a money iteration value
    ret_list = []
    players_remaining = copy.deepcopy(nPlayers)
    i = -1
    centers = copy.deepcopy(limits["C"])
    wings = copy.deepcopy(limits["W"])
    defenders = copy.deepcopy(limits["D"])
    goalies = copy.deepcopy(limits["G"])
    
    ## we loop through our table until our value (signifying the current item) is at 0
    while players_remaining > 0:
        
        ## If our traceback table tells us to buy the investment
        if combos_traceback[(centers, wings, defenders, goalies )][players_remaining][i]:
            
            ## We record the player position
            player_position = players[players_remaining - 1][3]
            
            ## We add our player's name, cost, ROI, and position to our output list
            output_row = (players[players_remaining - 1][0], players[players_remaining - 1][1] * inc, players[players_remaining - 1][2], players[players_remaining - 1][3])
            ret_list.append(output_row)
                            
            ## We increment our money iteration value backwards by the amount that we "spent" on the investment
            ## And increment our item marker value back by 1
            ## and incrememnt back the roster space
            i -= players[players_remaining - 1][1]
            players_remaining -= 1
            
            if player_position == "C":
                centers -= 1
            if player_position == "W":
                wings -= 1    
            if player_position == "D":
                defenders -= 1
            elif player_position == "G":
                goalies -= 1

        
        ## If our traceback table tells us to not buy the investment,
        ## we simply increment our item marker value back by 1
        else:
            players_remaining -= 1
    
    
    return ret_list#, combos, combos_traceback
 
    

In [13]:
limits = {"C": 2,"W": 3 ,"D" : 2,"G": 1}
results = get_optimal_set(data_for_algo, 50000, 190, limits)

In [14]:
results

[('Samuel Girard', 3800, 9.95, 'D'),
 ('Devon Toews', 4750, 11.2, 'D'),
 ('Nazem Kadri', 4940, 12.08, 'C'),
 ('Viktor Arvidsson', 5320, 12.69, 'W'),
 ('Anze Kopitar', 6840, 14.73, 'C'),
 ('Zach Fucale', 7600, 24.7, 'G'),
 ('Patrick Kane', 7790, 17.8, 'W'),
 ('Alex Ovechkin', 8930, 20.73, 'W')]

In [21]:
def calculate_results(results):
    cap = 0
    points = 0
    for player in results:
        cap += player[1]
        points += player[2]
    
    return (cap, points)

In [22]:
calculate_results(results)

(49970, 123.88)

# Part 2: Drafting from the middle

This is a modified version of our drafting algorithm that is meant for a mid-draft scenareo. This factors in players that you have already selected and players that have been selected by other people.

In [18]:
def get_player_info(players, target):
    for player in players:
        if player[0] == target:
            return(player)

def draft_from_middle(players, already_selected, players_taken_by_other, budget, inc, limits):
    
    ## for the already selected players, we adjust our budget and 
    for selected_player in already_selected:
        player_info = get_player_info(players, selected_player)
        budget = budget - player_info[1]
        limits[player_info[3]] -= 1
    
    
    ## we remove the already selected and taken players from our player pool
    remaining_players = [player for player in players if player[0] not in players_taken_by_other]    
    remaining_players = [player for player in remaining_players if player[0] not in already_selected]    
    
    ## and use our original drafting algorithm on the updated budget, limits, and player pool
    remaining_draft = get_optimal_set(remaining_players, budget, inc, limits)
    
    ## at the very end, we add in our already selected players
    for already_selected_player in already_selected:
        remaining_draft += [get_player_info(players, already_selected_player)]
    
    return remaining_draft
    

In [19]:
players = players_df[["Name", "Salary", "AvgPointsPerGame", "Position"]].values.tolist()

already_selected = ['Nathan MacKinnon']
players_taken_by_other = ['David Pastrnak', 'Patrice Bergeron']
limits = {"C": 2,"W": 3 ,"D" : 2,"G": 1}


mid_draft = draft_from_middle(players, already_selected, players_taken_by_other, 50000, 100, limits)

In [23]:
calculate_results(mid_draft)

(49800, 119.11)

# Part 3: ranking optimized list of player by order of importance

Now that we can get an optimized draft that includes players already selected and players that are already taken, we are going to create an updated version of our drafting algorithm

it will:
 - run a preliminary optimized draft on all of the available players
 - for every player selected in our preliminary result, we will:
     - run another optimized draft as if the player was unavailable
     - When we remove the selected player, our "optimal" result will have a somewhat lower total points scored.
     - We subtract this smaller total points from our optimal preliminary draft to get the player's draft contribution
        - draft contribution = how many points a player adds to your optimal solution
 - we rank each player by their contribution, we should first draft the player with the highest draft contribution

In [26]:
def get_player_importance(players, already_selected, players_taken_by_other, budget, inc, limits, verbose):
    
    #run a preliminary optimized draft on all of the available players
    results = draft_from_middle(players, already_selected, players_taken_by_other, budget, inc, limits)
    optimal_points = calculate_results(results)[1]

    players_with_contributions = []

    #for every player selected in our preliminary result, we will:
    for player in results:
    
        #run anohter optimized draft as if the player was unavailable
        already_selected = []
        updated_players_taken_by_other = players_taken_by_other + [player[0]]
        updated_reuslts = draft_from_middle(players, already_selected, updated_players_taken_by_other, 50000, 100, limits)
        max_points_without_player = calculate_results(updated_reuslts)[1]


        #we subtract it from our original optimized draft to get the players contribution
        player_contribution = optimal_points - max_points_without_player
        
        if verbose: 
            print("Player:",player[0],"-- Position:", player[3],"-- Contribution Value:", player_contribution)
            
        players_with_contributions.append((player[0], optimal_points, max_points_without_player, player_contribution))

    #we rank each player by their contribution, we should first draft the player with the highest draft contribution
    players_ranked = sorted(players_with_contributions, key=lambda x: x[3], reverse = True)
    
    top_player = players_ranked[0]
    print('You should draft:', top_player[0])
    
    return players_ranked

In [27]:
already_selected = []
players_taken_by_other = []
limits = {"C": 2,"W": 3 ,"D" : 2,"G": 1}

start_time = time.time()
top_players = get_player_importance(players, already_selected, players_taken_by_other, 50000, 100, limits, verbose = True)
print("--- %s seconds ---" % (time.time() - start_time))

Player: Erik Johnson -- Position: D -- Contribution Value: 0.06999999999999318
Player: Devon Toews -- Position: D -- Contribution Value: 0.06999999999999318
Player: Nazem Kadri -- Position: C -- Contribution Value: 0.11999999999999034
Player: Alex Iafallo -- Position: W -- Contribution Value: 0.06999999999999318
Player: Anze Kopitar -- Position: C -- Contribution Value: 0.4099999999999966
Player: Zach Fucale -- Position: G -- Contribution Value: 7.290000000000006
Player: Patrick Kane -- Position: W -- Contribution Value: 1.8599999999999994
Player: Alex Ovechkin -- Position: W -- Contribution Value: 2.469999999999999
You should draft: Zach Fucale
--- 65.1597957611084 seconds ---


In [28]:
top_players

[('Zach Fucale', 121.89, 114.6, 7.290000000000006),
 ('Alex Ovechkin', 121.89, 119.42, 2.469999999999999),
 ('Patrick Kane', 121.89, 120.03, 1.8599999999999994),
 ('Anze Kopitar', 121.89, 121.48, 0.4099999999999966),
 ('Nazem Kadri', 121.89, 121.77000000000001, 0.11999999999999034),
 ('Erik Johnson', 121.89, 121.82000000000001, 0.06999999999999318),
 ('Devon Toews', 121.89, 121.82000000000001, 0.06999999999999318),
 ('Alex Iafallo', 121.89, 121.82000000000001, 0.06999999999999318)]

# Part 4

Applying our Draft Algorithm to Scenareos where I have a "Flex" Player (the default in DraftKings).

In other words, I am allowed to add an additional player, regardless of the player's position

The solution is pretty simple. We run the algorithm 3 times (limits + 1 center, limits + 1 winger, limits + 1 defender). And we pick the solution with the highest amount. Since DraftKings does not do any live drafting (no worries about someone else taking our players), we don't need to draft from the middle.


In [43]:
def draft_with_util(data_for_algo, cap, inc, limits):
   
    best_combo = ''
    highest_points = -1
    util_draft = ''
    
    ## If you get one extra center
    
    limits_extra_center = copy.deepcopy(limits)
    limits_extra_center["C"] += 1
    results_extra_center = get_optimal_set(data_for_algo, cap, inc, limits_extra_center)
    total_points_extra_center = sum([player[2] for player in results_extra_center])
   
    if total_points_extra_center > highest_points:
        highest_points = total_points_extra_center
        best_combo = results_extra_center
        util_draft = "Center"
    
    print('Extra Center:', total_points_extra_center, 'expected points')

    ## if you get one extra Wing    
    limits_extra_wing = copy.deepcopy(limits)
    limits_extra_wing["W"] += 1
    results_extra_wing = get_optimal_set(data_for_algo, cap, inc, limits_extra_wing)
    total_points_extra_wing = sum([player[2] for player in results_extra_wing])

    if total_points_extra_wing > highest_points:
        highest_points = total_points_extra_wing
        best_combo = results_extra_wing
        util_draft = "Wing"

    print('Extra Wing:', total_points_extra_wing, 'expected points')

    ## if you get one extra Defender    
    limits_extra_defender = copy.deepcopy(limits)
    limits_extra_defender["D"] += 1
    
    results_extra_defender = get_optimal_set(data_for_algo, cap, inc, limits_extra_defender)
    total_points_extra_defender = sum([player[2] for player in results_extra_defender])

    if total_points_extra_defender > highest_points:
        highest_points = total_points_extra_defender
        best_combo = results_extra_defender
        util_draft = "Defender"

    print('Extra Defender:', total_points_extra_defender, 'expected points')


    ## We get the best option from our 
    best_option = max(total_points_dict, key=total_points_dict.get)
    print("The Util should be used for an aditional", util_draft)
    return best_combo, highest_points


In [44]:
limits = {"C": 2,"W": 3 ,"D" : 2,"G": 1}
draft_with_util(data_for_algo, 50000, 100, limits)

Extra Center: 126.07 expected points
Extra Wing: 126.38999999999999 expected points
Extra Defender: 125.38 expected points
The Util should be used for an aditional Wing


([('Brandon Tanev', 2700, 7.99, 'W'),
  ('Samuel Girard', 3800, 9.95, 'D'),
  ("Logan O'Connor", 3900, 9.65, 'W'),
  ('Devon Toews', 4800, 11.2, 'D'),
  ('Nazem Kadri', 5100, 12.08, 'C'),
  ('J.T. Miller', 5200, 12.29, 'C'),
  ('Zach Fucale', 7600, 24.7, 'G'),
  ('Patrick Kane', 7800, 17.8, 'W'),
  ('Alex Ovechkin', 9100, 20.73, 'W')],
 126.38999999999999)

# Part 5 Draft with the same setup as DraftKings (1 util, IR list)

Finally, Draft Kings provides you with a list of players that are listed on the injury reserve. This version preemptively 

In [59]:
def draft_with_util_and_ir(data_for_algo, cap, inc, limits, ir_list):
    remaining_players = [player for player in data_for_algo if player[0] not in ir_list]    
    results = draft_with_util(remaining_players, cap, inc, limits)
    return results
    

In [60]:
limits = {"C": 2,"W": 3 ,"D" : 2,"G": 1}
ir_list = ['Nathan MacKinnon','Pavel Francouz','Drew Doughty',"Zach Fucale"]
draft_with_util_and_ir(data_for_algo,50000,100,limits, ir_list)

Extra Center: 118.78 expected points
Extra Wing: 119.1 expected points
Extra Defender: 118.09 expected points
The Util should be used for an aditional Wing


([('Brandon Tanev', 2700, 7.99, 'W'),
  ('Samuel Girard', 3800, 9.95, 'D'),
  ("Logan O'Connor", 3900, 9.65, 'W'),
  ('Devon Toews', 4800, 11.2, 'D'),
  ('Nazem Kadri', 5100, 12.08, 'C'),
  ('J.T. Miller', 5200, 12.29, 'C'),
  ('Jonathan Quick', 7600, 17.41, 'G'),
  ('Patrick Kane', 7800, 17.8, 'W'),
  ('Alex Ovechkin', 9100, 20.73, 'W')],
 119.1)